# 📊 서울시 편의점 매출 완벽 통계분석
## "왜 어떤 편의점은 잘 팔리고 어떤 곳은 안 팔릴까?"

---

### 🎯 분석 목표
- **종속변수(Y)**: 편의점 매출
- **독립변수(X)**: 유동인구, 점포수, 상권유형(골목/발달/전통시장/관광특구)

### 📋 분석 목차

**[기본 분석]**
1. 데이터 로드 및 병합
2. 상관분석 (유동인구 많으면 매출 높아?)
3. 다중회귀분석 (뭐가 제일 영향 커?)
4. VIF 다중공선성 (변수들 겹치는 거 없어?)
5. 잔차분석 (모델 믿을 만해?)
6. 더빈-왓슨 검정 (자기상관 없어?)
7. Cook's Distance (이상한 데이터 있어?)
8. ANOVA 분산분석 (상권유형별 매출 차이 있어?)
9. Tukey 사후검정 (어디랑 어디가 차이나?)
10. 변수선택법 (최적 변수 조합은?)
11. 회귀계수 신뢰구간 (회귀계수 얼마나 확실해?)

**[심화 분석]**
12. 표준화 회귀계수 (진짜 영향력 순위는?)
13. 효과 크기 (실제로 얼마나 큰 차이야?)
14. 교호작용 분석 (유동인구 효과가 상권마다 달라?)
15. 시계열 추세 (연도별로 어떻게 변했어?)

**[결론]**
16. 최종 결론 및 발표 스토리

---
## 1. 라이브러리 및 데이터 로드

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor, OLSInfluence
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from sklearn.preprocessing import StandardScaler
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

print("✅ 라이브러리 로드 완료!")

In [ ]:
# === 매출 데이터 로드 ===
df_sales_2022 = pd.read_csv('data/sales_2022.csv', encoding='cp949')
df_sales_2023 = pd.read_csv('data/sales_2023.csv', encoding='cp949')
df_sales_2024 = pd.read_csv('data/sales_2024.csv', encoding='cp949')
df_sales = pd.concat([df_sales_2022, df_sales_2023, df_sales_2024], ignore_index=True)
df_sales = df_sales[df_sales['서비스_업종_코드_명'] == '편의점'].copy()
print(f"✅ 매출 데이터: {len(df_sales):,}개")

# === 점포수 데이터 로드 ===
df_store_2022 = pd.read_csv('data/store_2022.csv', encoding='cp949')
df_store_2023 = pd.read_csv('data/store_2023.csv', encoding='cp949')
df_store_2024 = pd.read_csv('data/store_2024.csv', encoding='cp949')
df_store = pd.concat([df_store_2022, df_store_2023, df_store_2024], ignore_index=True)
df_store = df_store[df_store['서비스_업종_코드_명'] == '편의점'].copy()
df_store = df_store[['기준_년분기_코드', '행정동_코드', '점포_수']]
print(f"✅ 점포 데이터: {len(df_store):,}개")

# === 유동인구 데이터 로드 ===
df_pop = pd.read_csv('data/floating_population.csv', encoding='cp949')
df_pop = df_pop[df_pop['기준_년분기_코드'].between(20221, 20244)].copy()
df_pop = df_pop[['기준_년분기_코드', '행정동_코드', '총_유동인구_수']]
print(f"✅ 유동인구 데이터: {len(df_pop):,}개")

# === 상권유형 데이터 로드 ===
df_type = pd.read_csv('data/district_type.csv', encoding='cp949')
df_type_pivot = df_type.groupby(['행정동_코드', '상권_구분_코드_명']).size().unstack(fill_value=0).reset_index()
df_type_pivot.columns.name = None
print(f"✅ 상권유형 데이터: {len(df_type_pivot):,}개 행정동")
print(f"\n📋 상권유형 컬럼: {[col for col in df_type_pivot.columns if col != '행정동_코드']}")

In [ ]:
# === 데이터 병합 ===
df = pd.merge(df_sales, df_store, on=['기준_년분기_코드', '행정동_코드'], how='inner')
print(f"매출 + 점포: {len(df):,}개")

df = pd.merge(df, df_pop, on=['기준_년분기_코드', '행정동_코드'], how='inner')
print(f"+ 유동인구: {len(df):,}개")

df = pd.merge(df, df_type_pivot, on='행정동_코드', how='left')
df = df.fillna(0)
print(f"+ 상권유형: {len(df):,}개")

# 분석용 데이터프레임 생성
df_analysis = df[['기준_년분기_코드', '행정동_코드', '행정동_코드_명',
                  '당월_매출_금액', '점포_수', '총_유동인구_수']].copy()

# 상권유형 컬럼 추가 (관광특구 포함!)
for col in ['골목상권', '발달상권', '전통시장', '관광특구']:
    if col in df.columns:
        df_analysis[col] = df[col]
    else:
        df_analysis[col] = 0

# 컬럼명 간소화
df_analysis = df_analysis.rename(columns={
    '당월_매출_금액': '매출',
    '점포_수': '점포수',
    '총_유동인구_수': '유동인구'
})

print(f"\n✅ 최종 분석 데이터: {len(df_analysis):,}개")
print(f"컬럼: {df_analysis.columns.tolist()}")

In [ ]:
# === 데이터 정합성 검증 ===
print("📋 데이터 정합성 검증")
print("=" * 50)
print(f"\n[데이터 규모]")
print(f"  - 총 데이터 수: {len(df_analysis):,}개")
print(f"  - 행정동 수: {df_analysis['행정동_코드'].nunique()}개")
print(f"  - 분기 수: {df_analysis['기준_년분기_코드'].nunique()}개")

print(f"\n[결측치 확인]")
null_counts = df_analysis.isnull().sum()
if null_counts.sum() == 0:
    print("  ✅ 결측치 없음!")
else:
    print(null_counts[null_counts > 0])

print(f"\n[주요 변수 기술통계]")
print(f"  - 매출 평균: {df_analysis['매출'].mean()/1e8:.2f}억원")
print(f"  - 매출 중앙값: {df_analysis['매출'].median()/1e8:.2f}억원")
print(f"  - 유동인구 평균: {df_analysis['유동인구'].mean()/1e6:.2f}백만명")
print(f"  - 점포수 평균: {df_analysis['점포수'].mean():.1f}개")

print(f"\n[상권유형별 데이터 분포]")
for col in ['골목상권', '발달상권', '전통시장', '관광특구']:
    nonzero = (df_analysis[col] > 0).sum()
    print(f"  - {col}: {nonzero:,}개 ({nonzero/len(df_analysis)*100:.1f}%)")

---
## 2. 상관분석 📈
### 💡 질문: "유동인구 많으면 매출 높아?"

**분석 목적**: 변수들 간의 관계 강도와 방향 확인

**해석 기준**:
- |r| > 0.7: 강한 상관
- |r| 0.4~0.7: 중간 상관
- |r| < 0.4: 약한 상관

In [ ]:
# 상관계수 계산 (관광특구 포함)
corr_cols = ['매출', '유동인구', '점포수', '골목상권', '발달상권', '전통시장', '관광특구']
corr_matrix = df_analysis[corr_cols].corr()

print("📊 상관계수 행렬")
print("=" * 80)
print(corr_matrix.round(3))

In [ ]:
# 히트맵 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='RdYlBu_r', center=0, fmt='.3f',
            square=True, linewidths=0.5, cbar_kws={'shrink': 0.8})
plt.title('변수 간 상관관계 히트맵', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# 매출과 각 변수의 상관관계 + 유의성 검정
print("📊 매출과 각 변수의 상관관계 (유의성 검정 포함)")
print("=" * 70)
print(f"{'변수':<12} {'상관계수':>10} {'p-value':>15} {'유의성':>8} {'해석':>15}")
print("-" * 70)

for col in ['유동인구', '점포수', '골목상권', '발달상권', '전통시장', '관광특구']:
    corr, p_val = stats.pearsonr(df_analysis['매출'], df_analysis[col])
    sig = "***" if p_val < 0.001 else "**" if p_val < 0.01 else "*" if p_val < 0.05 else "ns"
    
    if abs(corr) > 0.7:
        strength = "강한 상관"
    elif abs(corr) > 0.4:
        strength = "중간 상관"
    else:
        strength = "약한 상관"
    
    print(f"{col:<12} {corr:>10.4f} {p_val:>15.2e} {sig:>8} {strength:>15}")

print("\n*** p<0.001, ** p<0.01, * p<0.05, ns: 유의하지 않음")

---
## 3. 다중회귀분석 📉
### 💡 질문: "유동인구, 점포수, 상권유형 중 뭐가 제일 영향 커?"

**분석 목적**: 여러 독립변수가 매출에 미치는 영향 동시 분석

**해석 기준**:
- R²: 모델 설명력 (0~1, 높을수록 좋음)
- p-value < 0.05: 해당 변수가 유의미함
- 계수(coef): 1단위 증가 시 매출 변화량

In [ ]:
# 다중회귀분석 실행 (관광특구 포함)
X_cols = ['유동인구', '점포수', '골목상권', '발달상권', '전통시장', '관광특구']
X = df_analysis[X_cols]
X = sm.add_constant(X)
y = df_analysis['매출']

model = sm.OLS(y, X).fit()

print("📊 다중회귀분석 결과")
print("=" * 80)
print(model.summary())

In [ ]:
# 회귀계수 해석 (단위 변환)
print("\n📈 회귀계수 해석 (쉬운 해석)")
print("=" * 70)
print(f"\n모델 설명력 (R²): {model.rsquared:.4f} → 매출 변동의 {model.rsquared*100:.1f}%를 설명")
print(f"모델 유의성 (F-test p-value): {model.f_pvalue:.2e} → {'유의함' if model.f_pvalue < 0.05 else '유의하지 않음'}")

print("\n[변수별 영향력]")
print("-" * 70)
for var in X_cols:
    coef = model.params[var]
    pval = model.pvalues[var]
    sig = "✅ 유의함" if pval < 0.05 else "❌ 유의하지 않음"
    
    if var == '유동인구':
        effect = coef * 1e6 / 1e8
        print(f"{var}: 100만명 증가 시 매출 {effect:+.2f}억원 (p={pval:.4f}) {sig}")
    elif var == '점포수':
        effect = coef / 1e8
        print(f"{var}: 1개 증가 시 매출 {effect:+.2f}억원 (p={pval:.4f}) {sig}")
    else:
        effect = coef / 1e8
        print(f"{var}: 1개 증가 시 매출 {effect:+.2f}억원 (p={pval:.4f}) {sig}")

In [ ]:
# 회귀계수 시각화
coef_df = pd.DataFrame({
    '변수': X_cols,
    '계수(억원)': [model.params[col]/1e8 if col != '유동인구' else model.params[col]*1e6/1e8 for col in X_cols],
    'p-value': [model.pvalues[col] for col in X_cols]
})

plt.figure(figsize=(10, 6))
colors = ['green' if p < 0.05 else 'gray' for p in coef_df['p-value']]
bars = plt.barh(coef_df['변수'], coef_df['계수(억원)'], color=colors)
plt.axvline(x=0, color='black', linestyle='-', linewidth=1)
plt.xlabel('회귀계수 (억원 단위)')
plt.title('변수별 회귀계수\n(녹색=유의함 p<0.05, 회색=유의하지않음)', fontsize=12)
plt.tight_layout()
plt.show()

---
## 4. VIF 다중공선성 검정 🔍
### 💡 질문: "독립변수들끼리 겹치는 거 없어?"

**분석 목적**: 독립변수 간 상관이 너무 높으면 회귀계수 신뢰 불가

**해석 기준**:
- VIF < 5: 문제 없음 ✅
- VIF 5~10: 주의 필요 ⚠️
- VIF > 10: 심각한 다중공선성 ❌

In [ ]:
# VIF 계산
X_vif = df_analysis[X_cols]

vif_data = pd.DataFrame()
vif_data['변수'] = X_cols
vif_data['VIF'] = [variance_inflation_factor(X_vif.values, i) for i in range(len(X_cols))]
vif_data['판정'] = vif_data['VIF'].apply(lambda x: '❌ 심각' if x > 10 else '⚠️ 주의' if x > 5 else '✅ 양호')

print("📊 다중공선성 검정 (VIF)")
print("=" * 50)
print(vif_data.to_string(index=False))

In [ ]:
# VIF 시각화
plt.figure(figsize=(10, 6))
colors = ['red' if v > 10 else 'orange' if v > 5 else 'green' for v in vif_data['VIF']]
plt.barh(vif_data['변수'], vif_data['VIF'], color=colors)
plt.axvline(x=5, color='orange', linestyle='--', linewidth=2, label='VIF=5 (주의)')
plt.axvline(x=10, color='red', linestyle='--', linewidth=2, label='VIF=10 (심각)')
plt.xlabel('VIF')
plt.title('다중공선성 검정 (VIF)', fontsize=14, fontweight='bold')
plt.legend()
plt.tight_layout()
plt.show()

---
## 5. 잔차분석 📐
### 💡 질문: "우리 모델 믿을 만해?"

**분석 목적**: 회귀모델의 가정(정규성, 등분산성) 충족 여부 확인

**확인 항목**:
- 정규성: 잔차가 정규분포를 따르는가? (Shapiro-Wilk)
- 등분산성: 잔차 분산이 일정한가? (Breusch-Pagan)

In [ ]:
# 잔차 계산
residuals = model.resid
fitted = model.fittedvalues

# 4개 잔차 플롯
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

axes[0, 0].scatter(fitted/1e8, residuals/1e8, alpha=0.3, s=10)
axes[0, 0].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[0, 0].set_xlabel('적합값 (억원)')
axes[0, 0].set_ylabel('잔차 (억원)')
axes[0, 0].set_title('① 잔차 vs 적합값 (등분산성)')

axes[0, 1].hist(residuals/1e8, bins=50, edgecolor='black', alpha=0.7, density=True)
x_range = np.linspace(residuals.min()/1e8, residuals.max()/1e8, 100)
axes[0, 1].plot(x_range, stats.norm.pdf(x_range, residuals.mean()/1e8, residuals.std()/1e8), 'r-', linewidth=2)
axes[0, 1].set_xlabel('잔차 (억원)')
axes[0, 1].set_title('② 잔차 분포 (정규성)')

stats.probplot(residuals, dist="norm", plot=axes[1, 0])
axes[1, 0].set_title('③ Q-Q Plot (정규성)')

axes[1, 1].plot(residuals/1e8, alpha=0.5, linewidth=0.5)
axes[1, 1].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[1, 1].set_xlabel('관측치 순서')
axes[1, 1].set_ylabel('잔차 (억원)')
axes[1, 1].set_title('④ 잔차 vs 순서 (독립성)')

plt.tight_layout()
plt.show()

In [ ]:
# 정규성 & 등분산성 검정
sample_size = min(5000, len(residuals))
sample_residuals = residuals.sample(sample_size, random_state=42)
shapiro_stat, shapiro_p = stats.shapiro(sample_residuals)
bp_stat, bp_p, _, _ = het_breuschpagan(residuals, X)

print("📊 잔차 가정 검정")
print("=" * 60)
print(f"\n[정규성 검정 - Shapiro-Wilk]")
print(f"  p-value: {shapiro_p:.4e}")
print(f"  결과: {'✅ 정규성 만족' if shapiro_p >= 0.05 else '⚠️ 정규성 불만족 (but 데이터 많으면 OK)'}")

print(f"\n[등분산성 검정 - Breusch-Pagan]")
print(f"  p-value: {bp_p:.4e}")
print(f"  결과: {'✅ 등분산성 만족' if bp_p >= 0.05 else '⚠️ 이분산성 존재'}")

---
## 6. 더빈-왓슨 검정 🔄
### 💡 질문: "분기별 데이터가 서로 영향 안 줘?"

**분석 목적**: 잔차 간 자기상관 여부 확인

**해석 기준**:
- DW ≈ 2: 자기상관 없음 ✅
- DW < 1.5: 양의 자기상관 ⚠️
- DW > 2.5: 음의 자기상관 ⚠️

In [ ]:
# 더빈-왓슨 검정
dw_stat = durbin_watson(residuals)

print("📊 더빈-왓슨 검정 (자기상관)")
print("=" * 50)
print(f"DW 통계량: {dw_stat:.4f}")
print()
if 1.5 <= dw_stat <= 2.5:
    print(f"결과: ✅ 자기상관 없음 (DW={dw_stat:.2f} ≈ 2)")
elif dw_stat < 1.5:
    print(f"결과: ⚠️ 양의 자기상관 (DW={dw_stat:.2f} < 1.5)")
else:
    print(f"결과: ⚠️ 음의 자기상관 (DW={dw_stat:.2f} > 2.5)")

In [ ]:
# 더빈-왓슨 시각화
plt.figure(figsize=(10, 3))
plt.axvspan(0, 1.5, alpha=0.3, color='red', label='양의 자기상관')
plt.axvspan(1.5, 2.5, alpha=0.3, color='green', label='자기상관 없음')
plt.axvspan(2.5, 4, alpha=0.3, color='orange', label='음의 자기상관')
plt.axvline(x=dw_stat, color='blue', linewidth=4, label=f'DW = {dw_stat:.2f}')
plt.xlim(0, 4)
plt.xlabel('Durbin-Watson 통계량')
plt.title('더빈-왓슨 검정 결과', fontsize=12)
plt.legend(loc='upper right')
plt.tight_layout()
plt.show()

---
## 7. Cook's Distance 🎯
### 💡 질문: "결과 왜곡하는 이상한 데이터 있어?"

**분석 목적**: 회귀 결과에 과도한 영향을 미치는 이상치 탐지

**해석 기준**:
- Cook's D > 4/n: 영향력 있는 관측치
- Cook's D > 1: 매우 영향력 큼 (제거 고려)

In [ ]:
# Cook's Distance 계산
influence = OLSInfluence(model)
cooks_d = influence.cooks_distance[0]
n = len(df_analysis)
threshold = 4 / n
influential_idx = np.where(cooks_d > threshold)[0]

print("📊 Cook's Distance 분석")
print("=" * 50)
print(f"임계값 (4/n): {threshold:.6f}")
print(f"영향력 있는 관측치: {len(influential_idx)}개 ({len(influential_idx)/n*100:.1f}%)")
print(f"Cook's D > 1인 관측치: {sum(cooks_d > 1)}개")

In [ ]:
# Cook's Distance 시각화
plt.figure(figsize=(12, 5))
plt.stem(range(len(cooks_d)), cooks_d, markerfmt=',', basefmt=' ')
plt.axhline(y=threshold, color='red', linestyle='--', linewidth=2, label=f'임계값 = {threshold:.4f}')
plt.xlabel('관측치 인덱스')
plt.ylabel("Cook's Distance")
plt.title("Cook's Distance (이상치 탐지)", fontsize=12)
plt.legend()
plt.tight_layout()
plt.show()

---
## 8. ANOVA 분산분석 📊
### 💡 질문: "골목 vs 발달 vs 전통시장 vs 관광특구, 매출 차이 있어?"

**분석 목적**: 4개 그룹 간 평균 차이 검정

**해석 기준**:
- p-value < 0.05: 그룹 간 유의한 차이 있음

In [ ]:
# 주요 상권유형 결정
def get_main_type(row):
    types = {'골목상권': row['골목상권'], '발달상권': row['발달상권'], 
             '전통시장': row['전통시장'], '관광특구': row['관광특구']}
    if max(types.values()) == 0:
        return '기타'
    return max(types, key=types.get)

df_analysis['주요_상권유형'] = df_analysis.apply(get_main_type, axis=1)

# 기술통계
print("📊 상권유형별 매출 기술통계")
print("=" * 60)
type_stats = df_analysis.groupby('주요_상권유형')['매출'].agg(['count', 'mean', 'std'])
type_stats['mean'] = type_stats['mean'] / 1e8
type_stats['std'] = type_stats['std'] / 1e8
type_stats.columns = ['개수', '평균(억원)', '표준편차']
print(type_stats.round(2))

In [ ]:
# 박스플롯
plt.figure(figsize=(12, 6))
order = ['발달상권', '관광특구', '골목상권', '전통시장', '기타']
available_types = [t for t in order if t in df_analysis['주요_상권유형'].unique()]
sns.boxplot(data=df_analysis, x='주요_상권유형', y='매출', order=available_types)
plt.title('상권유형별 매출 분포', fontsize=14)
plt.ylabel('매출 (원)')
plt.tight_layout()
plt.show()

In [ ]:
# ANOVA
groups = [group['매출'].values for name, group in df_analysis.groupby('주요_상권유형')]
f_stat, anova_p = stats.f_oneway(*groups)

print("📊 ANOVA 분산분석 결과")
print("=" * 50)
print(f"F-통계량: {f_stat:.4f}")
print(f"p-value: {anova_p:.4e}")
print()
if anova_p < 0.05:
    print("결과: ✅ 상권유형별 매출에 유의한 차이가 있음")
else:
    print("결과: ❌ 유의한 차이 없음")

---
## 9. Tukey 사후검정 🔬
### 💡 질문: "정확히 어디랑 어디가 차이나?"

**분석 목적**: 어떤 그룹 간에 차이가 있는지 확인

In [ ]:
# Tukey HSD
tukey_result = pairwise_tukeyhsd(df_analysis['매출'], df_analysis['주요_상권유형'], alpha=0.05)
print("📊 Tukey HSD 사후검정 결과")
print("=" * 80)
print(tukey_result)

In [ ]:
# Tukey 시각화
fig, ax = plt.subplots(figsize=(12, 8))
tukey_result.plot_simultaneous(ax=ax)
plt.title('Tukey HSD 사후검정 (95% 신뢰구간)\n→ 겹치지 않으면 유의한 차이', fontsize=12)
plt.tight_layout()
plt.show()

---
## 10. 변수선택법 🎛️
### 💡 질문: "최적의 변수 조합은 뭐야?"

**분석 목적**: 가장 설명력 높은 변수 조합 찾기

In [ ]:
# 모든 변수 조합 비교
all_vars = ['유동인구', '점포수', '골목상권', '발달상권', '전통시장', '관광특구']
results = []

for k in range(1, len(all_vars) + 1):
    for combo in combinations(all_vars, k):
        X_temp = sm.add_constant(df_analysis[list(combo)])
        model_temp = sm.OLS(y, X_temp).fit()
        results.append({
            '변수': ', '.join(combo),
            '변수개수': k,
            'Adj_R²': model_temp.rsquared_adj,
            'AIC': model_temp.aic
        })

results_df = pd.DataFrame(results).sort_values('Adj_R²', ascending=False)

print("📊 변수 조합별 모델 성능 (상위 10개)")
print("=" * 80)
print(results_df.head(10).to_string(index=False))

In [ ]:
# 최적 모델
best_vars = results_df.iloc[0]['변수']
best_adj_r2 = results_df.iloc[0]['Adj_R²']

print(f"\n🏆 최적 모델: {best_vars}")
print(f"Adjusted R²: {best_adj_r2:.4f} ({best_adj_r2*100:.1f}%)")

---
## 11. 회귀계수 신뢰구간 📏
### 💡 질문: "회귀계수가 얼마나 확실해?"

**해석**: 95% 신뢰구간에 0이 포함되면 유의하지 않음

In [ ]:
# 신뢰구간
conf_int = model.conf_int(alpha=0.05)
conf_int.columns = ['하한(2.5%)', '상한(97.5%)']
conf_int['계수'] = model.params
conf_int['p-value'] = model.pvalues

print("📊 회귀계수 95% 신뢰구간")
print("=" * 70)
print(conf_int[['계수', '하한(2.5%)', '상한(97.5%)', 'p-value']].round(4))

In [ ]:
# 신뢰구간 시각화
fig, ax = plt.subplots(figsize=(12, 7))

for i, var in enumerate(X_cols):
    scale = 1e6/1e8 if var == '유동인구' else 1/1e8
    coef = model.params[var] * scale
    ci_low = conf_int.loc[var, '하한(2.5%)'] * scale
    ci_high = conf_int.loc[var, '상한(97.5%)'] * scale
    
    color = 'green' if model.pvalues[var] < 0.05 else 'gray'
    ax.plot([ci_low, ci_high], [i, i], color=color, linewidth=3)
    ax.scatter(coef, i, color=color, s=100, zorder=5)

ax.axvline(x=0, color='red', linestyle='--', linewidth=2)
ax.set_yticks(range(len(X_cols)))
ax.set_yticklabels(X_cols)
ax.set_xlabel('회귀계수 (억원 단위)')
ax.set_title('회귀계수 95% 신뢰구간\n(녹색=유의, 빨간선=0)', fontsize=12)
plt.tight_layout()
plt.show()

---
## 12. 표준화 회귀계수 (Beta 계수) 📊
### 💡 질문: "유동인구랑 점포수 중에 뭐가 진짜 더 영향 커?"

**문제점**: 일반 회귀계수는 단위가 달라서 직접 비교 불가

**해결**: 표준화하면 단위 상관없이 비교 가능! |β|가 클수록 영향력 큼

In [ ]:
# 표준화 회귀분석
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(df_analysis[X_cols]), columns=X_cols)
y_scaled = (df_analysis['매출'] - df_analysis['매출'].mean()) / df_analysis['매출'].std()

X_scaled_const = sm.add_constant(X_scaled)
model_std = sm.OLS(y_scaled, X_scaled_const).fit()

# 결과 정리
beta_df = pd.DataFrame({
    '변수': X_cols,
    '표준화 계수(β)': [model_std.params[col] for col in X_cols],
    '|β|': [abs(model_std.params[col]) for col in X_cols],
    'p-value': [model_std.pvalues[col] for col in X_cols]
}).sort_values('|β|', ascending=False)

print("📊 표준화 회귀계수 - 영향력 순위")
print("=" * 60)
print(beta_df.to_string(index=False))

In [ ]:
# 표준화 계수 시각화
plt.figure(figsize=(10, 6))
beta_sorted = beta_df.sort_values('표준화 계수(β)')
colors = ['green' if p < 0.05 else 'gray' for p in beta_sorted['p-value']]
plt.barh(beta_sorted['변수'], beta_sorted['표준화 계수(β)'], color=colors)
plt.axvline(x=0, color='black', linestyle='-', linewidth=1)
plt.xlabel('표준화 회귀계수 (β)')
plt.title('📊 표준화 회귀계수 비교 (|β| 클수록 영향력 큼)', fontsize=12)
plt.tight_layout()
plt.show()

print("\n🏆 매출 영향력 순위:")
for rank, (_, row) in enumerate(beta_df.iterrows(), 1):
    print(f"  {rank}위: {row['변수']} (β={row['표준화 계수(β)']:.4f})")

---
## 13. 효과 크기 (Effect Size) 📏
### 💡 질문: "통계적으로 유의한 건 알겠는데, 실제로 얼마나 큰 차이야?"

**η² (에타 제곱)**: 상권유형이 매출 변동의 몇 %를 설명하는지
- η² < 0.01: 작은 효과
- η² 0.01~0.06: 중간 효과
- η² > 0.14: 큰 효과

In [ ]:
# 에타 제곱 계산
def calculate_eta_squared(data, group_col, value_col):
    groups = data.groupby(group_col)[value_col]
    grand_mean = data[value_col].mean()
    ss_between = sum(len(g) * (g.mean() - grand_mean)**2 for _, g in groups)
    ss_total = sum((data[value_col] - grand_mean)**2)
    return ss_between / ss_total

eta_squared = calculate_eta_squared(df_analysis, '주요_상권유형', '매출')

print("📊 ANOVA 효과 크기 (η²)")
print("=" * 50)
print(f"η² = {eta_squared:.4f} ({eta_squared*100:.2f}%)")

if eta_squared < 0.01:
    effect_size = "작은 효과"
elif eta_squared < 0.06:
    effect_size = "중간 효과"
else:
    effect_size = "큰 효과"

print(f"\n해석: 상권유형이 매출 변동의 {eta_squared*100:.2f}%를 설명 → {effect_size}")

In [ ]:
# Cohen's d
def cohens_d(g1, g2):
    n1, n2 = len(g1), len(g2)
    pooled_std = np.sqrt(((n1-1)*g1.var() + (n2-1)*g2.var()) / (n1+n2-2))
    return (g1.mean() - g2.mean()) / pooled_std

print("\n📊 Cohen's d (두 그룹 간 효과 크기)")
print("=" * 60)
print("|d| < 0.2 작음, 0.2~0.8 중간, > 0.8 큼")
print("-" * 60)

types = df_analysis['주요_상권유형'].unique()
for i, t1 in enumerate(types):
    for t2 in types[i+1:]:
        g1 = df_analysis[df_analysis['주요_상권유형'] == t1]['매출']
        g2 = df_analysis[df_analysis['주요_상권유형'] == t2]['매출']
        d = cohens_d(g1, g2)
        size = "작음" if abs(d) < 0.2 else "중간" if abs(d) < 0.8 else "큼"
        print(f"{t1} vs {t2}: d={d:.3f} ({size})")

---
## 14. 교호작용 분석 🔀
### 💡 질문: "유동인구 효과가 상권유형에 따라 다를까?"

예: 발달상권에서 유동인구 100만↑ → 매출 +10억
    골목상권에서 유동인구 100만↑ → 매출 +3억

In [ ]:
# 교호작용 모델
df_inter = df_analysis.copy()
for col in ['골목상권', '발달상권', '전통시장', '관광특구']:
    df_inter[f'유동인구×{col}'] = df_inter['유동인구'] * df_inter[col]

inter_cols = X_cols + ['유동인구×골목상권', '유동인구×발달상권', '유동인구×전통시장', '유동인구×관광특구']
X_inter = sm.add_constant(df_inter[inter_cols])
model_inter = sm.OLS(df_inter['매출'], X_inter).fit()

print("📊 교호작용 분석 결과")
print("=" * 60)
print(f"기존 R²: {model.rsquared:.4f} → 교호작용 R²: {model_inter.rsquared:.4f}")
print(f"R² 증가: +{(model_inter.rsquared - model.rsquared)*100:.2f}%")

print("\n[교호작용 항 유의성]")
for col in ['유동인구×골목상권', '유동인구×발달상권', '유동인구×전통시장', '유동인구×관광특구']:
    coef = model_inter.params[col]
    pval = model_inter.pvalues[col]
    sig = "✅" if pval < 0.05 else "❌"
    effect = coef * 1e6 / 1e8
    print(f"  {col}: {effect:+.2f}억원 (p={pval:.4f}) {sig}")

In [ ]:
# 상권유형별 유동인구 효과 시각화 (상권 포함 지역 기준으로 수정)
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for ax, stype, color in zip(axes.flat, ['발달상권', '골목상권', '전통시장', '관광특구'], 
                            ['blue', 'orange', 'green', 'red']):
    # 핵심 수정: '주요_상권유형' 대신 해당 상권이 1개 이상 있는 지역 필터링
    mask = df_analysis[stype] > 0
    data = df_analysis[mask]
    
    if len(data) > 10:
        ax.scatter(data['유동인구']/1e6, data['매출']/1e8, alpha=0.3, s=10, color=color)
        z = np.polyfit(data['유동인구']/1e6, data['매출']/1e8, 1)
        p = np.poly1d(z)
        x_line = np.linspace(data['유동인구'].min()/1e6, data['유동인구'].max()/1e6, 100)
        ax.plot(x_line, p(x_line), 'r-', linewidth=2)
        ax.set_title(f'{stype} 포함 지역 (n={len(data):,})\n기울기={z[0]:.1f}억원/백만명', fontsize=11)
    else:
        ax.text(0.5, 0.5, f'{stype}\n데이터 부족 (n={len(data)})', 
               ha='center', va='center', fontsize=14, transform=ax.transAxes)
        ax.set_title(f'{stype}')
    ax.set_xlabel('유동인구 (백만명)')
    ax.set_ylabel('매출 (억원)')

plt.suptitle('📊 상권유형별 유동인구 → 매출 효과 비교 (교호작용)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print('\n💡 해석: 기울기가 클수록 유동인구 효과가 큼')

---
## 15. 시계열 추세 분석 📈
### 💡 질문: "2022년 → 2024년, 매출 패턴이 어떻게 바뀌었어?"

In [ ]:
# 연도/분기 컬럼
df_analysis['연도'] = df_analysis['기준_년분기_코드'].astype(str).str[:4].astype(int)
df_analysis['분기'] = df_analysis['기준_년분기_코드'].astype(str).str[4:].astype(int)

# 연도별 통계
print("📊 연도별 평균 매출")
print("=" * 40)
yearly = df_analysis.groupby('연도')['매출'].mean()
for year, sales in yearly.items():
    print(f"{year}년: {sales/1e8:.2f}억원")

In [ ]:
# 시계열 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. 분기별 추이
quarterly = df_analysis.groupby('기준_년분기_코드')['매출'].mean()
axes[0,0].plot(quarterly.index.astype(str), quarterly.values/1e8, marker='o', linewidth=2)
axes[0,0].set_title('① 분기별 평균 매출', fontsize=12)
axes[0,0].tick_params(axis='x', rotation=45)
axes[0,0].grid(True, alpha=0.3)

# 2. 연도별 비교
colors = ['#3498db', '#2ecc71', '#e74c3c']
bars = axes[0,1].bar(yearly.index.astype(str), yearly.values/1e8, color=colors[:len(yearly)])
axes[0,1].set_title('② 연도별 평균 매출', fontsize=12)
for bar, val in zip(bars, yearly.values/1e8):
    axes[0,1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3, f'{val:.1f}', ha='center')

# 3. 상권유형별 연도 추이 (수정: 상권 포함 지역 기준)
for stype in ['발달상권', '골목상권', '전통시장', '관광특구']:
    mask = df_analysis[stype] > 0  # 핵심 수정!
    if mask.sum() > 100:  # 데이터 충분할 때만
        yr = df_analysis[mask].groupby('연도')['매출'].mean()
        axes[1,0].plot(yr.index, yr.values/1e8, marker='o', label=stype, linewidth=2)
axes[1,0].set_title('③ 상권유형별 연도 추이', fontsize=12)
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# 4. 성장률 (수정: 상권 포함 지역 기준)
growth = {}
for stype in ['발달상권', '골목상권', '전통시장', '관광특구']:
    mask = df_analysis[stype] > 0  # 핵심 수정!
    if mask.sum() > 100:
        yr = df_analysis[mask].groupby('연도')['매출'].mean()
        if len(yr) >= 2:
            growth[stype] = (yr.iloc[-1] - yr.iloc[0]) / yr.iloc[0] * 100

if growth:
    growth_df = pd.DataFrame({'상권': growth.keys(), '성장률(%)': growth.values()})
    growth_df = growth_df.sort_values('성장률(%)')
    colors = ['green' if g > 0 else 'red' for g in growth_df['성장률(%)']]
    axes[1,1].barh(growth_df['상권'], growth_df['성장률(%)'], color=colors)
    axes[1,1].axvline(x=0, color='black')
    axes[1,1].set_title('④ 상권유형별 성장률 (2022→2024)', fontsize=12)
    for i, (_, row) in enumerate(growth_df.iterrows()):
        axes[1,1].text(row['성장률(%)'] + 0.2, i, f\"{row['성장률(%)']:.1f}%\", va='center')

plt.tight_layout()
plt.show()

print('\n📊 성장률 요약:')
print(growth_df.to_string(index=False))

---
# 📝 16. 최종 결론 및 발표 스토리

In [ ]:
# 유의한 변수
sig_vars = [var for var in X_cols if model.pvalues[var] < 0.05]
nonsig_vars = [var for var in X_cols if model.pvalues[var] >= 0.05]

print('='*80)
print('📝 서울시 편의점 매출 통계분석 최종 결론')
print('='*80)

print('''
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃  🎯 연구 질문: "왜 어떤 편의점은 잘 팔리고 어떤 곳은 안 팔릴까?"      ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
''')

print('\n' + '━'*80)
print('📊 [1] 기본 분석 결과 요약')
print('━'*80)

print(f'''
┌─────────────────────────────────────────────────────────────────────┐
│ 📈 다중회귀분석                                                      │
├─────────────────────────────────────────────────────────────────────┤
│   • 모델 설명력 (R²): {model.rsquared:.4f} ({model.rsquared*100:.1f}%)                            │
│   • 유의한 변수: {', '.join(sig_vars) if sig_vars else '없음'}                                   │
│   • 유의하지 않은 변수: {', '.join(nonsig_vars) if nonsig_vars else '없음'}                        │
└─────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────┐
│ ✅ 모델 검증                                                         │
├─────────────────────────────────────────────────────────────────────┤
│   • VIF (다중공선성): {'✅ 문제없음' if vif_data['VIF'].max() < 5 else '⚠️ 주의필요'}                                │
│   • 더빈-왓슨 (자기상관): DW = {dw_stat:.2f} {'✅ 문제없음' if 1.5 <= dw_stat <= 2.5 else '⚠️ 주의'}              │
│   • Cook's D (이상치): {len(influential_idx):,}개 ({len(influential_idx)/n*100:.1f}%)                              │
└─────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────┐
│ 📊 ANOVA (상권유형별 차이)                                           │
├─────────────────────────────────────────────────────────────────────┤
│   • F = {f_stat:.2f}, p-value = {anova_p:.2e}                                       │
│   • 결론: {'✅ 상권유형별 매출에 유의한 차이 있음' if anova_p < 0.05 else '❌ 유의한 차이 없음'}                   │
└─────────────────────────────────────────────────────────────────────┘
''')

In [ ]:
# 심화 분석 결과 + 비즈니스 인사이트
print('\n' + '━'*80)
print('📊 [2] 심화 분석 결과')
print('━'*80)

print('''
┌─────────────────────────────────────────────────────────────────────┐
│ 📊 표준화 회귀계수 (영향력 순위)                                     │
├─────────────────────────────────────────────────────────────────────┤''')
for rank, (_, row) in enumerate(beta_df.head(4).iterrows(), 1):
    sig = '✅' if row['p-value'] < 0.05 else ''
    print(f"│   {rank}위: {row['변수']:<10} (β = {row['표준화 계수(β)']:>7.4f}) {sig:<20}│")
print('└─────────────────────────────────────────────────────────────────────┘')

print(f'''
┌─────────────────────────────────────────────────────────────────────┐
│ 📏 효과 크기 (η²)                                                    │
├─────────────────────────────────────────────────────────────────────┤
│   • η² = {eta_squared:.4f}                                                       │
│   • 해석: 상권유형이 매출 변동의 {eta_squared*100:.2f}%를 설명 → {effect_size}           │
└─────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────┐
│ 🔀 교호작용 분석                                                     │
├─────────────────────────────────────────────────────────────────────┤
│   • 기존 R²: {model.rsquared:.4f} → 교호작용 R²: {model_inter.rsquared:.4f}                     │
│   • R² 증가: +{(model_inter.rsquared-model.rsquared)*100:.2f}%                                              │
│   • 의미: 유동인구 효과가 상권유형에 따라 다르게 나타남              │
└─────────────────────────────────────────────────────────────────────┘
''')

if len(growth_df) > 0:
    best_growth = growth_df.iloc[-1]
    worst_growth = growth_df.iloc[0]
    print(f'''
┌─────────────────────────────────────────────────────────────────────┐
│ 📈 시계열 추세 (2022→2024)                                           │
├─────────────────────────────────────────────────────────────────────┤
│   • 가장 높은 성장: {best_growth['상권']:<8} ({best_growth['성장률(%)']:>+6.1f}%)                      │
│   • 가장 낮은 성장: {worst_growth['상권']:<8} ({worst_growth['성장률(%)']:>+6.1f}%)                      │
└─────────────────────────────────────────────────────────────────────┘
''')

print('\n' + '━'*80)
print('💡 [3] 핵심 인사이트 & 비즈니스 제언')
print('━'*80)

print(f'''
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 🎯 핵심 발견                                                        ┃
┣━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┫
┃                                                                     ┃
┃  1️⃣ 매출에 가장 큰 영향: {beta_df.iloc[0]['변수']:<30}          ┃
┃                                                                     ┃
┃  2️⃣ 상권유형별 매출: 발달상권 > 관광특구 > 골목상권 > 전통시장     ┃
┃                                                                     ┃
┃  3️⃣ 유동인구 효과는 상권마다 다름                                   ┃
┃     → 발달상권/관광특구에서 유동인구 효과가 더 큼                   ┃
┃     → 골목상권은 유동인구 효과가 상대적으로 작음                    ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 📌 전략적 제언                                                      ┃
┣━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┫
┃                                                                     ┃
┃  🏪 신규 출점 전략                                                  ┃
┃  ──────────────────────────────────────────────────────────────    ┃
┃  • 우선순위: 발달상권 > 관광특구 > 전통시장 > 골목상권              ┃
┃  • 핵심: 유동인구가 많고, 매출 전환율이 높은 입지 선택              ┃
┃                                                                     ┃
┃  🏪 기존 점포 운영 전략                                             ┃
┃  ──────────────────────────────────────────────────────────────    ┃
┃  • 발달상권/관광특구: 유동인구 극대화 (좋은 위치, 가시성 확보)      ┃
┃  • 골목상권: 유동인구 의존도 낮추기 (배달, 정기구독, 단골확보)      ┃
┃  • 전통시장: 관광객 + 지역주민 동시 타겟팅                          ┃
┃                                                                     ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
''')

In [ ]:
# 요약 대시보드 시각화
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# 1. R² 게이지
ax1 = axes[0, 0]
ax1.pie([model.rsquared, 1-model.rsquared], labels=[f'설명력\n{model.rsquared*100:.1f}%', ''], 
        colors=['#2ecc71', '#ecf0f1'], startangle=90, wedgeprops=dict(width=0.4))
ax1.set_title('① 모델 설명력 (R²)', fontsize=12, fontweight='bold')

# 2. 표준화 계수 Top 3
ax2 = axes[0, 1]
beta_top3 = beta_df.head(3)
colors = ['#3498db', '#9b59b6', '#e67e22']
ax2.barh(beta_top3['변수'], beta_top3['|β|'], color=colors)
ax2.set_xlabel('|β|')
ax2.set_title('② 영향력 Top 3', fontsize=12, fontweight='bold')

# 3. 상권유형별 평균 매출
ax3 = axes[0, 2]
type_mean = df_analysis.groupby('주요_상권유형')['매출'].mean().sort_values(ascending=True) / 1e8
colors = plt.cm.RdYlGn(np.linspace(0.2, 0.8, len(type_mean)))
ax3.barh(type_mean.index, type_mean.values, color=colors)
ax3.set_xlabel('평균 매출 (억원)')
ax3.set_title('③ 상권유형별 평균 매출', fontsize=12, fontweight='bold')

# 4. VIF
ax4 = axes[1, 0]
colors = ['green' if v < 5 else 'orange' if v < 10 else 'red' for v in vif_data['VIF']]
ax4.barh(vif_data['변수'], vif_data['VIF'], color=colors)
ax4.axvline(x=5, color='orange', linestyle='--')
ax4.set_xlabel('VIF')
ax4.set_title('④ 다중공선성 (VIF)', fontsize=12, fontweight='bold')

# 5. 성장률
ax5 = axes[1, 1]
if len(growth_df) > 0:
    colors = ['green' if g > 0 else 'red' for g in growth_df['성장률(%)']]
    ax5.barh(growth_df['상권'], growth_df['성장률(%)'], color=colors)
    ax5.axvline(x=0, color='black')
ax5.set_xlabel('성장률 (%)')
ax5.set_title('⑤ 상권유형별 성장률', fontsize=12, fontweight='bold')

# 6. 효과 크기
ax6 = axes[1, 2]
ax6.pie([eta_squared, 1-eta_squared], labels=[f'상권유형\n{eta_squared*100:.1f}%', '기타'], 
        colors=['#e74c3c', '#ecf0f1'], startangle=90, wedgeprops=dict(width=0.4))
ax6.set_title('⑥ 효과 크기 (η²)', fontsize=12, fontweight='bold')

plt.suptitle('📊 서울시 편의점 매출 분석 요약 대시보드', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# 발표 스크립트
print('\n' + '='*80)
print('🎤 발표 스크립트')
print('='*80)

best_var = beta_df.iloc[0]['변수']
best_growth_name = growth_df.iloc[-1]['상권'] if len(growth_df) > 0 else '발달상권'
best_growth_val = growth_df.iloc[-1]['성장률(%)'] if len(growth_df) > 0 else 0

print(f'''
"안녕하세요, 르네3's 팀입니다.

저희 팀은 '왜 어떤 편의점은 잘 팔리고 어떤 곳은 안 팔릴까?'라는 질문에 
답하기 위해 서울시 편의점 매출 데이터를 분석했습니다.

📊 [데이터]
서울 열린데이터광장에서 2022~2024년 편의점 매출, 유동인구, 점포수, 
상권유형 데이터를 수집해 약 {len(df_analysis):,}개의 데이터를 분석했습니다.

📊 [기본 분석 결과]
다중회귀분석 결과, 모델의 설명력 R²는 {model.rsquared*100:.1f}%였습니다.
통계적으로 유의한 변수는 {', '.join(sig_vars)}이었고,
ANOVA 분석에서 상권유형별 매출 차이도 통계적으로 유의했습니다.

📊 [심화 분석 결과]
표준화 회귀계수를 분석한 결과, 매출에 가장 큰 영향을 미치는 변수는 
'{best_var}'였습니다.

교호작용 분석 결과, 유동인구 효과는 상권유형에 따라 다르게 나타났습니다.
발달상권과 관광특구에서 유동인구 효과가 더 컸고, 
골목상권에서는 상대적으로 효과가 작았습니다.

시계열 분석 결과, 2022년 대비 2024년 매출 성장률이 가장 높은 상권은 
'{best_growth_name}' ({best_growth_val:+.1f}%)였습니다.

📌 [결론]
편의점 매출을 높이려면:
1. 발달상권/관광특구 입지 선택 (유동인구 효과 극대화)
2. 성장률 높은 상권 타겟팅
3. 골목상권은 배달/특화상품 등 차별화 전략 필요

감사합니다."
''')

---
## ✅ 분석 완료!

### 📋 전체 분석 목록 (16개)

**[기본 분석]**
1. 데이터 로드 | 2. 상관분석 | 3. 다중회귀 | 4. VIF | 5. 잔차 | 6. 더빈왓슨 | 7. Cook's D | 8. ANOVA | 9. Tukey | 10. 변수선택 | 11. 신뢰구간

**[심화 분석]**
12. 표준화 계수 | 13. 효과 크기 | 14. 교호작용 | 15. 시계열

**[결론]**
16. 최종 결론 및 비즈니스 인사이트

### 🎯 이 노트북의 강점
1. **완전한 통계 검증**: 가정 검정부터 효과 크기까지
2. **실무 해석**: 억원 단위 변환, 비즈니스 제언
3. **시각화**: 모든 분석에 그래프 포함
4. **발표 준비**: 요약 대시보드와 스크립트